# P3Alpha

### Import libraries

In [28]:
import pandas as pd
import numpy as np
from scipy import sparse
import os
import csv

from utilities import load_sparse_matrix

### Load dataset

In [29]:
URM = load_sparse_matrix("urm.npz")
URM_train = load_sparse_matrix("urm_train.npz")
URM_val = load_sparse_matrix("urm_val.npz")
ICM = load_sparse_matrix("icm.npz")

In [ ]:
from src.Evaluation.Evaluator import EvaluatorHoldout

evaluator_val = EvaluatorHoldout(URM_val, cutoff_list=[10])

EvaluatorHoldout: Ignoring 315 ( 0.8%) Users that have less than 1 test interactions


In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(5, 1000),
    "alpha": Real(0, 1),
}

In [ ]:
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = P3alphaRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class, evaluator_validation=evaluator_val)

In [ ]:
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [ ]:
hyperparameterSearch.search(
    recommender_input_args,
    hyperparameter_search_space = hyperparameters_range_dictionary,
    n_cases = n_cases,
    n_random_starts = n_random_starts,
    save_model = "last",
    output_folder_path = output_folder_path, # Where to save the results
    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
    metric_to_optimize = metric_to_optimize,
    cutoff_to_optimize = cutoff_to_optimize,
)

SearchBayesianSkopt: argument save_model is 'last' but no recommender_input_args_last_test provided, saving best model on train data alone.
Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 685, 'alpha': 0.567302821264038, 'similarity': 'cosine', 'normalize': False}
P3alphaRecommender: URM Detected 3461 (12.4%) items with no interactions.
SearchBayesianSkopt: Config 0 Exception. Config: {'topK': 685, 'alpha': 0.567302821264038, 'similarity': 'cosine', 'normalize': False} - Exception: Traceback (most recent call last):
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 484, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 321, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(
  

Traceback (most recent call last):
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 484, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 321, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 293, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
TypeError: P3alphaRecommender.fit() got an unexpected keyword argument 'similarity'


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.3024
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 142, 'alpha': 0.8314849632497184, 'similarity': 'cosine', 'normalize': True}
P3alphaRecommender: URM Detected 3461 (12.4%) items with no interactions.
SearchBayesianSkopt: Config 1 Exception. Config: {'topK': 142, 'alpha': 0.8314849632497184, 'similarity': 'cosine', 'normalize': True} - Exception: Traceback (most recent call last):
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 484, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 321, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(
  F

Traceback (most recent call last):
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 484, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 321, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 293, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
TypeError: P3alphaRecommender.fit() got an unexpected keyword argument 'similarity'


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.2684
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 3 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 823, 'alpha': 0.7854801961633652, 'similarity': 'cosine', 'normalize': False}
P3alphaRecommender: URM Detected 3461 (12.4%) items with no interactions.
SearchBayesianSkopt: Config 2 Exception. Config: {'topK': 823, 'alpha': 0.7854801961633652, 'similarity': 'cosine', 'normalize': False} - Exception: Traceback (most recent call last):
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 484, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 321, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(
 

Traceback (most recent call last):
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 484, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 321, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(
  File "/home/fra/Documents/university/recsys-challenge-2022/src/HyperparameterTuning/SearchAbstractClass.py", line 293, in _fit_model
    recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
TypeError: P3alphaRecommender.fit() got an unexpected keyword argument 'similarity'
